In [227]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
from python_functions.duplicate_functions import drop_dup_ext1, drop_dup_ext2

In [2]:
ext2 = pd.read_csv('Датасет\\db2\\Ext2.csv')
ext1 = pd.read_csv('Датасет\\db2\\Ext.csv')
catch = pd.read_csv('Датасет\\db1\\catch.csv') 
product = pd.read_csv('Датасет\\db1\\product.csv') 

### Общая предобработка

In [3]:
ext1.rename(columns={'id_fishery':'id_ves'}, inplace=True)
#ext1.loc[ext1.Product_period == '\\N', 'Product_period'] = np.NaN
# ext1.loc[ext1.Name_Plat == '\\N', 'Name_Plat'] = np.NaN
# ext1_mrg = ext1.merge(ext2, on='id_vsd', how='left')
catch.date = pd.to_datetime(catch.date)
product.date = pd.to_datetime(product.date)
ext1.date_fishery = pd.to_datetime(ext1.date_fishery) 
# ext1.Product_period = pd.to_datetime(ext1.Product_period) 
ext2.date_vsd = pd.to_datetime(ext2.date_vsd) 


### Возьмем данные за 10 последних дней

In [241]:
samp_1 = ext1[(ext1.date_fishery >= pd.to_datetime('2022-04-15')) & (ext1.date_fishery <= pd.to_datetime('2022-04-20'))]
samp_2 = ext2[(ext2.date_vsd >= pd.to_datetime('2022-04-15')) & (ext2.date_vsd <= pd.to_datetime('2022-04-20'))]

In [228]:
ext1_new = drop_dup_ext1(ext1)
ext2_new = drop_dup_ext2(ext2)

In [242]:
samp_2['abs_error'] = 0
tmp = samp_2[samp_2.duplicated('id_vsd', keep=False)]
tmp.sort_values(by=['id_vsd', 'date_vsd'])


<ipython-input-242-3252ea9ad0ab>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samp_2['abs_error'] = 0


,id_vsd,num_vsd,id_fish,fish,date_vsd,volume,unit,abs_error
2668792,610,3174359,714,камбала-ерш,2022-04-15 01:33:39,1718,\N,0
3082790,610,3174359,-1,камбала,2022-04-15 01:33:39,2,тонна,0
2668904,3715,2895714,408,пикша,2022-04-15 04:27:04,30,\N,0
3082997,3715,2895714,-1,пикша свежая,2022-04-15 04:27:04,0,тонна,0
2670456,4977,7962217,410,сайда,2022-04-18 03:44:36,1773,\N,0
...,...,...,...,...,...,...,...,...
3081541,9516941,1042304,-1,треска,2022-04-16 07:52:39,8,тонна,0
2669455,9519703,5894747,112,палтус белокорый,2022-04-16 03:54:56,68,\N,0
2781535,9519703,5894747,-1,палтус белокорый,2022-04-16 03:54:56,0,тонна,0
2668696,9519800,7658662,617,зубатка пестрая,2022-04-15 01:33:43,594,\N,0


In [274]:
def smart_dup(ext1, ext2):
    df2 = ext2.copy()
    df2['abs_error'] = 0
    initial_drop = df2[df2.duplicated(['id_vsd', 'fish'], keep=False)].index
    df2 = df2.drop(initial_drop)

    tmp = df2[df2.duplicated('id_vsd', keep=False)]

    drop_idx = tmp.sort_values(by=['id_vsd', 'unit']).index[1::2]
    # good_idx = tmp.sort_values(by=['id_vsd', 'unit']).index[::2]
    tmp = tmp.drop(drop_idx)

    df1 = ext1.copy()
    tmp1 = df1[df1.duplicated(['id_vsd'], keep=False)].sort_values(by=['id_vsd', 'id_ves'])
    change_idx = df1[df1.duplicated(['id_vsd'], keep=False)].sort_values(by=['id_vsd', 'id_ves']).index
    df1.loc[change_idx, 'Product_period'] = (tmp1.Product_period + ' - ' + tmp1.date_fishery.astype(str)).values
    df1.loc[change_idx[::2], 'id_ves'] = (df1.loc[change_idx[1::2], 'id_ves']).values

    drop_idx = df1[df1.duplicated(['id_vsd'], keep='first')].sort_values(by=['id_vsd', 'id_ves']).index
    df1 = df1.loc[drop_idx]

    return df1


# ext2.drop((tmp.loc[drop_idx, 'abs_error'] > 500).index).shape
# ext2 = ext2.drop((tmp.loc[drop_idx, 'abs_error'] > 500).index)
smart = smart_dup(ext1, ext2)


In [275]:
smart

,id_ves,id_own,date_fishery,numPart,id_Plat,id_vsd,Name_Plat,Product_period,Region_Plat
539959,2337731,4743567,2022-04-03,\N,2352176,71,\N,\N - 2022-04-03 00:00:00,\N
653036,1147871,4565284,2022-03-30,\N,743924,534,\N,\N - 2022-03-30 00:00:00,\N
200145,2181156,5132476,2022-04-14,\N,484587,610,\N,\N - 2022-04-14 00:00:00,\N
273341,86689,6644915,2022-04-12,\N,5904397,859,\N,\N - 2022-04-12 00:00:00,\N
313917,2402426,3846064,2022-04-11,\N,4485643,962,\N,\N - 2022-04-11 00:00:00,\N
...,...,...,...,...,...,...,...,...,...
2450063,6605677,8090733,2022-01-28,\N,9228181,9520496,\N,\N - 2022-01-28 00:00:00,\N
1671601,1930206,4760527,2022-02-23,\N,7698233,9520578,\N,\N - 2022-02-23 00:00:00,\N
1968241,4543391,2507472,2022-02-14,\N,2975925,9520583,\N,\N - 2022-02-14 00:00:00,\N
2401139,3200620,8379463,2022-01-30,\N,4646693,9520820,\N,\N - 2022-01-30 00:00:00,\N


In [262]:
smart

,id_vsd,num_vsd,id_fish,fish,date_vsd,volume,unit,abs_error
38610,71,5067251,714,камбала-ерш,2022-04-04 02:41:41,55,\N,0
2545983,71,5067251,-1,камбала,2022-04-04 02:41:41,0,тонна,0
2668792,610,3174359,714,камбала-ерш,2022-04-15 01:33:39,1718,\N,0
3082790,610,3174359,-1,камбала,2022-04-15 01:33:39,2,тонна,0
2667755,859,1320861,147,окунь золотистый,2022-04-13 01:52:15,230,\N,0
...,...,...,...,...,...,...,...,...
2543918,9520578,9096984,-1,палтус черный свежий,2022-02-24 06:14:34,2,тонна,0
18119,9520583,1920887,714,камбала-ерш,2022-02-15 16:39:59,125,\N,0
2540407,9520583,1920887,-1,камбала,2022-02-15 16:39:59,0,тонна,0
8441,9520874,2011831,617,зубатка пестрая,2022-01-23 02:33:04,1038,\N,0


In [244]:
ext2['abs_error'] = 0
initial_drop = ext2[ext2.duplicated(['id_vsd', 'fish'], keep=False)].index
ext2_test = ext2.drop(initial_drop)

tmp = ext2_test[ext2_test.duplicated('id_vsd', keep=False)]

# drop_idx = tmp.sort_values(by=['id_vsd', 'unit']).index[1::2]
# good_idx = tmp.sort_values(by=['id_vsd', 'unit']).index[::2]
# ext2.loc[drop_idx, 'abs_error'] = abs(ext2.loc[drop_idx, 'volume'].values * 1000 - ext2.loc[good_idx, 'volume'].values)
tmp.sort_values(by=['id_vsd', 'date_vsd'])
# ext2.drop((tmp.loc[drop_idx, 'abs_error'] > 500).index).shape
# ext2 = ext2.drop((tmp.loc[drop_idx, 'abs_error'] > 500).index)


,id_vsd,num_vsd,id_fish,fish,date_vsd,volume,unit,abs_error
38610,71,5067251,714,камбала-ерш,2022-04-04 02:41:41,55,\N,0
2545983,71,5067251,-1,камбала,2022-04-04 02:41:41,0,тонна,0
2668792,610,3174359,714,камбала-ерш,2022-04-15 01:33:39,1718,\N,0
3082790,610,3174359,-1,камбала,2022-04-15 01:33:39,2,тонна,0
2667755,859,1320861,147,окунь золотистый,2022-04-13 01:52:15,230,\N,0
...,...,...,...,...,...,...,...,...
2543918,9520578,9096984,-1,палтус черный свежий,2022-02-24 06:14:34,2,тонна,0
18119,9520583,1920887,714,камбала-ерш,2022-02-15 16:39:59,125,\N,0
2540407,9520583,1920887,-1,камбала,2022-02-15 16:39:59,0,тонна,0
8441,9520874,2011831,617,зубатка пестрая,2022-01-23 02:33:04,1038,\N,0


In [233]:
def find_catch_errors(samp_1, samp_2, catch):
    vsd_list = list(set(samp_1.id_vsd) & set(samp_2.id_vsd))
    vsd_samp1 = samp_1[samp_1.id_vsd.isin(vsd_list)]
    vsd_samp2 = samp_2[samp_2.id_vsd.isin(vsd_list)]
    db1_full = vsd_samp1.merge(vsd_samp2, on='id_vsd', how='left')
    own_list = list(set(db1_full.id_own) & set(catch.id_own))
    mrg_own = db1_full[db1_full.id_own.isin(own_list)]
    date_list = list(set(mrg_own.date_fishery) & set(catch.date))
    date_mrg_own = mrg_own[mrg_own.date_fishery.isin(date_list)].rename(columns={'date_fishery':'date'})
    catch_db1 = catch.merge(date_mrg_own, on=['date', 'id_own', 'id_fish']) \
        .drop(columns=['id_ves_y', 'abs_error', 'numPart', 'id_Plat', 'permit', 'Name_Plat']) \
            .rename(columns={'id_ves_x':'id_ves'})
    return catch_db1

test_df = find_catch_errors(ext1_new, ext2_new, catch)



,id_ves,date,id_region,id_fish,catch_volume,id_regime,id_own,id_vsd,Product_period,Region_Plat,num_vsd,fish,date_vsd,volume,unit
2,5177805,2022-01-01,269,400,216.914,12,9290322,2268589,\N,\N,6631026,минтай,2022-01-01 18:12:04,120000,\N
3,2701764,2022-01-04,265,451,12.550,41,7115834,2827406,\N,\N,8080333,камбалы дальневосточные,2022-01-05 01:08:07,15500,\N
4,2024908,2022-01-04,265,451,11.480,41,7115834,2827406,\N,\N,8080333,камбалы дальневосточные,2022-01-05 01:08:07,15500,\N
5,8270286,2022-01-04,264,451,4.815,41,7115834,2827406,\N,\N,8080333,камбалы дальневосточные,2022-01-05 01:08:07,15500,\N
6,2701764,2022-01-04,265,400,0.386,41,7115834,8690693,\N,\N,6278101,минтай,2022-01-05 01:08:07,297,\N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2070,6344959,2022-04-20,108,714,0.165,97,6808535,4635884,\N,\N,5946455,камбала-ерш,2022-04-21 09:49:18,214,\N
2077,16137,2022-04-20,206,617,1.011,97,6934839,8752935,\N,\N,398149,зубатка пестрая,2022-04-21 09:44:47,1010,\N
2085,3454068,2022-04-20,108,294,8.304,1,2391781,2690643,\N,\N,9149943,краб-стригун опилио,2022-04-21 09:47:10,1926,\N
2101,6443786,2022-04-20,116,409,42.660,15,1778753,3824083,\N,\N,7717261,путассу (северная),2022-04-21 09:47:02,40080,\N


In [ ]:
test_df[(test_df['volume'] / 1000 - test_df ['catch_volume']) != 0]

In [210]:
id_prod = pd.read_csv('Датасет\\db1\\ref\\prod_type.csv', sep=';')[['id_prod_type', 'prod_type']]

In [223]:
product_type = product[(product.date==pd.to_datetime('2022-04-16')) & (product.id_ves.isin([742971, 8379477]))] \
    .merge(id_prod, on='id_prod_type', how='left')
product_type

,id_ves,date,id_prod_designate,id_prod_type,prod_volume,prod_board_volume,prod_type
0,742971,2022-04-16,1,1450,0.00000,0.01600,фарш серый из брюшков минт мор
1,742971,2022-04-16,1,1473,0.00000,0.00600,икра минт ястыч мор с 5-6
2,742971,2022-04-16,1,2014,0.09120,0.09120,икра сельди яст мор
3,742971,2022-04-16,1,2509,0.00000,0.00400,мука корм рыб отх
4,742971,2022-04-16,1,3136,1.77750,1.77750,фл сельди т/о мор двойн
5,742971,2022-04-16,1,3501,0.00000,0.00600,минт фл б к мор б кс
6,742971,2022-04-16,1,3964,0.00000,0.00600,молоки минт мор
7,742971,2022-04-16,1,4555,48.20200,48.20200,сельдь т/о кр жр н/р мор
8,8379477,2022-04-16,1,2598,0.77500,3.32500,мука корм рыб
9,8379477,2022-04-16,1,10056,176.92556,1056.64407,СТАВРИДА ОК МОР Н/РАЗ1


In [226]:
product_type.merge(date_mrg_own, on=['date'])# \
   # .drop(columns=['id_fish_x','id_fish_y', 'id_ves_x', 'id_ves_y', 'abs_error'])

,id_ves_x,date,id_prod_designate,id_prod_type,prod_volume,prod_board_volume,prod_type,id_ves_y,id_own,numPart,...,Name_Plat,Product_period,Region_Plat,num_vsd,id_fish,fish,date_vsd,volume,unit,abs_error
0,742971,2022-04-16,1,1450,0.00000,0.01600,фарш серый из брюшков минт мор,6124052,7478978,\N,...,\N,\N,\N,3691111,204,сельдь тихоокеанская,2022-04-17 05:03:06,51822,\N,0
1,742971,2022-04-16,1,1473,0.00000,0.00600,икра минт ястыч мор с 5-6,6124052,7478978,\N,...,\N,\N,\N,3691111,204,сельдь тихоокеанская,2022-04-17 05:03:06,51822,\N,0
2,742971,2022-04-16,1,2014,0.09120,0.09120,икра сельди яст мор,6124052,7478978,\N,...,\N,\N,\N,3691111,204,сельдь тихоокеанская,2022-04-17 05:03:06,51822,\N,0
3,742971,2022-04-16,1,2509,0.00000,0.00400,мука корм рыб отх,6124052,7478978,\N,...,\N,\N,\N,3691111,204,сельдь тихоокеанская,2022-04-17 05:03:06,51822,\N,0
4,742971,2022-04-16,1,3136,1.77750,1.77750,фл сельди т/о мор двойн,6124052,7478978,\N,...,\N,\N,\N,3691111,204,сельдь тихоокеанская,2022-04-17 05:03:06,51822,\N,0
5,742971,2022-04-16,1,3501,0.00000,0.00600,минт фл б к мор б кс,6124052,7478978,\N,...,\N,\N,\N,3691111,204,сельдь тихоокеанская,2022-04-17 05:03:06,51822,\N,0
6,742971,2022-04-16,1,3964,0.00000,0.00600,молоки минт мор,6124052,7478978,\N,...,\N,\N,\N,3691111,204,сельдь тихоокеанская,2022-04-17 05:03:06,51822,\N,0
7,742971,2022-04-16,1,4555,48.20200,48.20200,сельдь т/о кр жр н/р мор,6124052,7478978,\N,...,\N,\N,\N,3691111,204,сельдь тихоокеанская,2022-04-17 05:03:06,51822,\N,0
8,8379477,2022-04-16,1,2598,0.77500,3.32500,мука корм рыб,6124052,7478978,\N,...,\N,\N,\N,3691111,204,сельдь тихоокеанская,2022-04-17 05:03:06,51822,\N,0
9,8379477,2022-04-16,1,10056,176.92556,1056.64407,СТАВРИДА ОК МОР Н/РАЗ1,6124052,7478978,\N,...,\N,\N,\N,3691111,204,сельдь тихоокеанская,2022-04-17 05:03:06,51822,\N,0


In [293]:
ext2[(ext2.volume == 2188) & (ext2.unit == 'тонна')]

,id_vsd,num_vsd,id_fish,fish,date_vsd,volume,unit
94771,5405316,7273837,-1,густера мороженая,2022-03-29 13:11:48,2188,тонна


In [201]:
date_mrg_own

,id_ves,id_own,date,numPart,id_Plat,id_vsd,Name_Plat,Product_period,Region_Plat,num_vsd,id_fish,fish,date_vsd,volume,unit,abs_error
108186,6124052,7478978,2022-04-16,\N,8160548,3653625,\N,\N,\N,3691111,204,сельдь тихоокеанская,2022-04-17 05:03:06,51822,\N,0


In [305]:
tmp.iloc[i:i+2].date_vsd.iloc[0], tmp.iloc[i:i+2].date_vsd.iloc[1]

(Timestamp('2022-03-28 04:46:07'), Timestamp('2022-03-28 04:46:07'))

In [307]:
Timedelta('0 days 00:00:00')

NameError: name 'Timedelta' is not defined

In [51]:
tmp = samp_2[samp_2.volume > 0].sort_values(by='volume')
lines = tmp.iloc[3:3+2]

In [52]:
lines

,id_vsd,num_vsd,id_fish,fish,date_vsd,volume,unit,abs_error
2783292,7935482,1309498,-1,кольца кальмара в кляре,2022-04-16 07:04:47,1,кг,0
2783296,2752190,5170480,-1,сельдь соленая,2022-04-16 07:04:47,1,кг,0


In [ ]:
import json

In [71]:
anomaly_dict

{'violation': ['{"id_vsd":{"3093668":5496794,"2951336":8354870},"date_vsd":{"3093668":1650006857000,"2951336":1650240484000}}',
  '{"id_vsd":{"2759008":2392276,"3080549":6196911},"date_vsd":{"2759008":1650184945000,"3080549":1649989790000}}',
  '{"id_vsd":{"2668950":6707691,"2781329":8835985},"date_vsd":{"2668950":1649991517000,"2781329":1649991798000}}',
  '{"id_vsd":{"2781307":8487193,"2670040":2093504},"date_vsd":{"2781307":1650168344000,"2670040":1650173485000}}',
  '{"id_vsd":{"2670816":4967027,"3049226":9340176},"date_vsd":{"2670816":1650336046000,"3049226":1650337342000}}',
  '{"id_vsd":{"2670403":1732995,"3049253":9177127},"date_vsd":{"2670403":1650250031000,"3049253":1650250947000}}',
  '{"id_vsd":{"3050040":2333775,"2669523":7831397},"date_vsd":{"3050040":1650093671000,"2669523":1650084856000}}',
  '{"id_vsd":{"3049230":6642545,"2670049":146214},"date_vsd":{"3049230":1650168011000,"2670049":1650180176000}}',
  '{"id_vsd":{"2669365":7747480,"2948789":570067},"date_vsd":{"26693

In [95]:
with open('test.txt', 'w', encoding='utf8') as f:
        print(f'Задержки в пределах допустимого:\n', file=f)
        for line in anomaly_dict['mistake']:
            tmp = json.loads(line)
            print(pd.DataFrame(tmp).reset_index(drop=True), file=f) 
        print(file=f)
        print('-' * 40, file=f)
        print(file=f)
        print(f'Задержки больше {1} часа(ов):\n', file=f)
        for line in anomaly_dict['violation']:
            tmp = json.loads(line)
            print(pd.DataFrame(tmp).reset_index(drop=True), file=f)

       

In [77]:
test = json.loads(line)

In [96]:
pd.DataFrame(test)

,id_vsd,date_vsd
2670433,9079062,1650251654000
2781331,165569,1650251447000


In [61]:
with open('result.json', 'w') as fp:
    json.dump(anomaly_dict, fp)

TypeError: Object of type DataFrame is not JSON serializable

In [70]:
with open('report_delays.json', 'w', encoding='utf8') as f:
    json.dump(anomaly_dict, f, ensure_ascii=False)

In [67]:
for key in anomaly_dict:
    for i in range(len(anomaly_dict[key])):
        anomaly_dict[key][i] = anomaly_dict[key][i].to_json()

In [55]:
# ext2[(ext2.volume == 2188) & (ext2.unit == 'кг')]
with open('report_delays.json', 'w', encoding='utf8') as f:
    anomaly_dict = {'violation':[], 'mistake':[]}
    tmp = samp_2[samp_2.volume > 0].sort_values(by='volume')
    for i in tqdm(range(tmp.shape[0] - 1)):
        lines = tmp.iloc[i:i+2]
        c1 = lines.volume.iloc[0] == lines.volume.iloc[1]
        c2 = lines.unit.iloc[0] != lines.unit.iloc[1]
        c3 = pd.Timedelta(lines.date_vsd.iloc[0] - lines.date_vsd.iloc[1]).seconds / 3600.0 < 1
        c4 = lines.fish.iloc[0] == lines.fish.iloc[1]
        if c1 & c2 & c3 & c4:
            # print(c1, c2, c3, c4) 
            anomaly_dict['mistake'].append(lines[['id_vsd', 'date_vsd']].to_json())          
            # print(lines[['id_vsd', 'date_vsd'].iloc[0])
            # print(lines[['id_vsd', 'date_vsd']].iloc[1])
        if (c1 & c2 & c4) and not c3:
            anomaly_dict['violation'].append(lines[['id_vsd', 'date_vsd']].to_json())          
    json.dump(anomaly_dict, f)

100%|██████████| 132616/132616 [02:07<00:00, 1037.16it/s]


TypeError: Object of type DataFrame is not JSON serializable

In [88]:
pd.DataFrame(json.loads(anomaly_dict['mistake'][0]))

,id_vsd,date_vsd
2670433,9079062,1650251654000
2781331,165569,1650251447000


In [249]:
ton_idx = ext2[ext2.unit == 'тонна'].index
ext2.loc[ton_idx]

,id_vsd,num_vsd,id_fish,fish,date_vsd,volume,unit,abs_error
94771,5405316,7273837,-1,густера мороженая,2022-03-29 13:11:48,2188,тонна,0
107902,4488272,7055462,-1,филе судака мороженое,2022-03-09 16:42:52,20,тонна,0
254378,1227657,8793958,-1,пелядь,2022-02-25 12:26:49,6,тонна,0
269218,7350225,9369478,-1,карась мороженый,2022-04-04 10:51:37,0,тонна,0
269220,9059526,4154769,-1,окунь мороженый,2022-04-04 10:51:37,0,тонна,0
...,...,...,...,...,...,...,...,...
3083830,8890850,7229351,-1,треска,2022-04-06 13:36:04,26,тонна,0
3083831,4933135,6695747,-1,камбала-ерш охлажденная,2022-04-06 13:36:05,1,тонна,0
3083833,6175541,3611984,-1,палтус,2022-04-06 13:36:05,0,тонна,0
3083837,8518574,509199,-1,зубатка свежая,2022-04-06 13:36:05,1,тонна,0


In [268]:
tmp = ext2[ext2.duplicated(['fish', 'volume', 'date_vsd'], keep=False)].sort_values(by=['date_vsd', 'volume'])
tmp


,id_vsd,num_vsd,id_fish,fish,date_vsd,volume,unit,abs_error
961884,1952005,2138203,-1,лосось натуральный,2022-01-02 01:21:33,1,кг,0
961885,8251392,7493666,-1,лосось натуральный,2022-01-02 01:21:33,1,кг,0
1723508,6886695,5071236,-1,изделия кулинарные рыбные,2022-01-02 01:22:05,2,кг,0
1723509,1907676,1460236,-1,изделия кулинарные рыбные,2022-01-02 01:22:05,2,кг,0
1723519,1413232,7702684,-1,изделия кулинарные рыбные,2022-01-02 01:22:05,4,кг,0
...,...,...,...,...,...,...,...,...
2841252,561759,660306,-1,сельдь соленая,2022-04-20 02:36:51,0,кг,0
2841253,558424,5743965,-1,сельдь соленая,2022-04-20 02:36:51,0,кг,0
2841254,5068567,2080052,-1,сельдь соленая,2022-04-20 02:36:51,0,кг,0
3106340,823729,2287068,457,корюшка азиатская зубастая,2022-05-06 12:43:34,402,\N,0


In [260]:
(tmp.unit == 'кг') | (tmp.unit == '\\N')

834151     True
2213807    True
2213828    True
2213831    True
2213840    True
           ... 
2913221    True
741127     True
671277     True
2911392    True
2911391    True
Name: unit, Length: 2866074, dtype: bool

3069426

In [113]:
#ext2[ext2.duplicated('id_vsd', keep=False)].sort_values(by=['id_vsd', 'date_vsd']).head(4)
#tmp = ext2[ext2.duplicated('id_vsd', keep=False)]
#tmp[(tmp.unit == 'тонна') & (tmp.id_fish == -1)]91708 
#tmp.sort_values(by=['id_vsd', 'unit'])  #keep = first
ext2.sort_values(by=['id_vsd', 'unit']).drop_duplicates('id_vsd', keep='first').shape[0]

3069426

In [163]:
ext1[ext1.duplicated(['id_vsd'], keep=False)].sort_values(by=['id_vsd', 'date_fishery']).head(4)

,id_ves,id_own,date_fishery,numPart,id_Plat,id_vsd,Name_Plat,Product_period,Region_Plat
539959,2337731,4743567,2022-04-03 00:00:00,\N,2352176,71,NaN,NaN,\N
523999,-1,5502023,2022-04-04 02:41:41,\N,2352176,71,858685,2022-04-03 00:00:00,Мурманская область
653036,1147871,4565284,2022-03-30 00:00:00,\N,743924,534,NaN,NaN,\N
617757,-1,5502023,2022-03-31 03:08:31,\N,743924,534,7635039,2022-03-30 00:00:00,Сахалинская область


In [217]:
ext1 = ext1.drop(drop_idx[1::2])

In [206]:
change_idx = ext1[ext1.duplicated(['id_vsd'], keep=False)].sort_values(by=['id_vsd', 'id_ves']).index
ext1[ext1.duplicated(['id_vsd'], keep=False)].sort_values(by=['id_vsd', 'id_ves'])

,id_ves,id_own,date_fishery,numPart,id_Plat,id_vsd,Name_Plat,Product_period,Region_Plat
523999,2337731.0,5502023,2022-04-04 02:41:41,\N,2352176,71,858685,2022-04-03 00:00:00,Мурманская область
539959,2337731.0,4743567,2022-04-03 00:00:00,\N,2352176,71,\N,\N,\N
617757,1147871.0,5502023,2022-03-31 03:08:31,\N,743924,534,7635039,2022-03-30 00:00:00,Сахалинская область
653036,1147871.0,4565284,2022-03-30 00:00:00,\N,743924,534,\N,\N,\N
161959,2181156.0,5502023,2022-04-15 01:33:39,\N,484587,610,5068711,\N,Мурманская область
...,...,...,...,...,...,...,...,...,...
1968241,4543391.0,2507472,2022-02-14 00:00:00,\N,2975925,9520583,\N,\N,\N
2368612,3200620.0,5502023,2022-01-31 10:36:30,\N,4646693,9520820,1878350,2022-01-30 00:00:00,Мурманская область
2401139,3200620.0,8379463,2022-01-30 00:00:00,\N,4646693,9520820,\N,\N,\N
2608611,2634670.0,5502023,2022-01-23 02:33:04,\N,8805052,9520874,6312932,2022-01-22 00:00:00,Мурманская область


In [212]:
tmp = ext1[ext1.duplicated(['id_vsd'], keep=False)].sort_values(by=['id_vsd', 'id_ves'])
change_idx = ext1[ext1.duplicated(['id_vsd'], keep=False)].sort_values(by=['id_vsd', 'id_ves']).index
ext1.loc[change_idx, 'Product_period'] = (tmp.Product_period + ' - ' + tmp.date_fishery.astype(str)).values


In [66]:
ext1[(ext1.id_own == 4565284) & (ext1.date_fishery == pd.to_datetime('2022-03-30'))]

,id_ves,id_own,date_fishery,numPart,id_Plat,id_vsd,Name_Plat,Product_period,Region_Plat
652968,9472820,4565284,2022-03-30,\N,685187,7525317,\N,\N,\N
653035,1875749,4565284,2022-03-30,\N,1707704,5600915,\N,\N,\N
653036,1147871,4565284,2022-03-30,\N,743924,534,\N,\N,\N
653037,2807166,4565284,2022-03-30,\N,1874,9235148,\N,\N,\N


In [74]:
ext2[(ext2.id_vsd == 7525317) & (ext2.date_vsd >= pd.to_datetime('2022-03-30'))]

,id_vsd,num_vsd,id_fish,fish,date_vsd,volume,unit
36732,7525317,4185775,859,креветка северная,2022-03-31 02:05:21,1076,\N
1517049,7525317,4185775,-1,креветка северная,2022-03-31 02:05:21,1,тонна


In [220]:
catch[(catch.id_own == 5502023)]

,id_ves,date,id_region,id_fish,catch_volume,id_regime,permit,id_own


In [ ]:
ext2.unit=='тонна'

In [85]:
dfx = pd.DataFrame(range(1, 6)) # 1, 2, 3, 4, 5
dfx = dfx.rolling(3).mean()

In [86]:
dfx

,0
0,NaN
1,NaN
2,2.0
3,3.0
4,4.0


In [152]:
ext2_simp_ = pd.read_csv('Упрощённый датасет\\db2\\Ext2.csv')
ext1_simp_ = pd.read_csv('Упрощённый датасет\\db2\\Ext.csv')
catch_simp_ = pd.read_csv('Упрощённый датасет\\db1\\catch.csv') 
product_simp_ = pd.read_csv('Упрощённый датасет\\db1\\product.csv') 

In [ ]:
ext1_mrg = ext1.merge(ext2, on='id_vsd', how='left')

In [155]:
ext2_simp_.columns

Index(['id_vsd', 'num_vsd', 'id_fish', 'fish', 'date_vsd', 'volume', 'unit'], dtype='object')

In [8]:
ext1.rename(columns={'id_fishery':'id_ves'}, inplace=True)
catch.date = pd.to_datetime(catch.date)
product.date = pd.to_datetime(product.date)
ext1_mrg.date_fishery = pd.to_datetime(ext1_mrg.date_fishery) 
ext1_mrg.date_vsd = pd.to_datetime(ext1_mrg.date_vsd) 

NameError: name 'ext1_mrg' is not defined

In [244]:
ext1cols = ['id_vsd', 'id_ves', 'id_own', 'date_fishery', 'Product_period']
ext2cols = ['id_vsd', 'date_vsd', 'volume', 'unit', 'fish']
catchcols = ['id_ves', 'date', 'catch_volume', 'id_own']
productcols = ['id_ves', 'date', 'id_prod_designate', 'id_prod_type', 'prod_volume', 'prod_board_volume']

In [157]:
ext2_simp = ext2_simp_[ext2cols]
ext1_simp = ext1_simp_[ext1cols]
catch_simp = catch_simp_[catchcols]
product_simp = product_simp_[productcols]

In [247]:
ext2 = ext2_[ext2cols]
ext1 = ext1_[ext1cols]
catch = catch_[catchcols]
product = product_[productcols]

In [99]:
ext1_simp.shape, ext2_simp.shape, catch_simp.shape, product_simp.shape

((3260798, 5), (3115280, 4), (85087, 4), (395071, 6))

### Здесь проверим несостыковки для тех кораблей, что есть во всех таблицах

In [248]:
# ext1_simp_mrg = ext1_simp.merge(ext2_simp, on='id_vsd', how='left')
ext1_mrg = ext1.merge(ext2, on='id_vsd', how='left')

In [249]:
# catch_simp.date = pd.to_datetime(catch_simp.date)
# product_simp.date = pd.to_datetime(product_simp.date)
# ext1_simp_mrg.date_fishery = pd.to_datetime(ext1_simp_mrg.date_fishery) 
# ext1_simp_mrg.date_vsd = pd.to_datetime(ext1_simp_mrg.date_vsd) 

catch.date = pd.to_datetime(catch.date)
product.date = pd.to_datetime(product.date)
ext1_mrg.date_fishery = pd.to_datetime(ext1_mrg.date_fishery) 
ext1_mrg.date_vsd = pd.to_datetime(ext1_mrg.date_vsd) 


c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [250]:
# ext1_simp_mrg.rename(columns={'date_fishery':'date'}, inplace=True)
ext1_mrg.rename(columns={'date_fishery':'date'}, inplace=True)

In [254]:
# idx = list(set(catch_simp.id_ves) & set(product_simp.id_ves) & set(ext1_simp_mrg.id_ves))
idx_orig = list(set(catch.id_ves) & set(ext1_mrg.id_ves))

In [210]:
# errors = []
# for ves in tqdm(idx):
#     a1 = catch_simp[catch_simp.id_ves == ves]
#     a1['first_dig'] = a1.catch_volume.astype(str).str.slice(0, 2)
#     a2 = ext1_simp_mrg[ext1_simp_mrg.id_ves == ves]
#     a2 = a2.drop_duplicates(['date', 'id_ves', 'id_vsd'], 'first')
#     a2['first_dig'] = a2.volume.astype(str).str.slice(0, 2)

#     df1 = a1.merge(a2, on=['id_ves', 'date', 'first_dig'], how='right')
#     kilo_idx = df1[df1.unit == '\\N'].index
#     df1.loc[kilo_idx, 'volume'] = df1.volume / 1000
#     df1 = df1.dropna()
#     df1['error'] = 1 - df1.catch_volume / df1.volume
#     if not df1[df1['error']>0.05].empty:
#         errors.append(df1[df1['error']>0.05])
    

  0%|          | 0/326 [00:00<?, ?it/s]<ipython-input-210-3c705a00e097>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a1['first_dig'] = a1.catch_volume.astype(str).str.slice(0, 2)
<ipython-input-210-3c705a00e097>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a1['first_dig'] = a1.catch_volume.astype(str).str.slice(0, 2)
<ipython-input-210-3c705a00e097>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [261]:
list(set(catch.id_ves) & set(ext1.id_ves))

[]

In [252]:
errors = []
for ves in tqdm(idx_orig):
    a1 = catch[catch.id_ves == ves]
    a1['first_dig'] = a1.catch_volume.astype(str).str.slice(0, 2)
    a2 = ext1_mrg[ext1_mrg.id_ves == ves]
    a2 = a2.drop_duplicates(['date', 'id_ves', 'id_vsd'], 'first')
    a2['first_dig'] = a2.volume.astype(str).str.slice(0, 2)

    df1 = a1.merge(a2, on=['id_ves', 'date', 'first_dig'], how='right')
    kilo_idx = df1[df1.unit == '\\N'].index
    df1.loc[kilo_idx, 'volume'] = df1.volume / 1000
    df1 = df1.dropna()
    df1['error'] = 1 - df1.catch_volume / df1.volume
    if not df1[df1['error']>0.05].empty:
        errors.append(df1[df1['error']>0.05])
    

0it [00:00, ?it/s]


In [230]:
errors[0].date.values[0]

numpy.datetime64('2022-03-08T00:00:00.000000000')

In [231]:
catch_simp[(catch_simp.id_ves == errors[0].id_ves.values[0]) & (catch_simp.date==errors[0].date.values[0])]

,id_ves,date,catch_volume,id_own
48493,1286,2022-03-08,11.535,1091
48494,1286,2022-03-08,114.239,1091


In [238]:
for i in range(len(errors)):
    len_ext = ext1_simp_mrg[(ext1_simp_mrg.id_ves == errors[i].id_ves.values[0]) & (ext1_simp_mrg.date==errors[i].date.values[0])].shape[0]
    len_catch = catch_simp[(catch_simp.id_ves == errors[i].id_ves.values[0]) & (catch_simp.date==errors[i].date.values[0])].shape[0]
    if len_ext // 2 != len_catch:
        print(i)


4
2
0
------------------------------
10
5
1
------------------------------
10
5
2
------------------------------
10
5
3
------------------------------
4
2
4
------------------------------
4
2
5
------------------------------
8
4
6
------------------------------
8
4
7
------------------------------
4
2
8
------------------------------
8
4
9
------------------------------
4
2
10
------------------------------


In [135]:
catch_simp[catch_simp.id_ves == 1628]

,id_ves,date,catch_volume,id_own
0,1628,2022-01-01,31.874,961
376,1628,2022-01-02,80.147,961
2107,1628,2022-01-05,10.149,961
2728,1628,2022-01-06,208.182,961
3373,1628,2022-01-07,150.686,961
...,...,...,...,...
64062,1628,2022-03-28,120.047,961
64836,1628,2022-03-29,111.789,961
65565,1628,2022-03-30,196.385,961
66360,1628,2022-03-31,14.902,961


In [204]:
# a1 = catch_simp[catch_simp.id_ves == 1284]
# a1['first_dig'] = a1.catch_volume.astype(str).str.slice(0, 2)
# a2 = ext1_simp_mrg[ext1_simp_mrg.id_ves == 1284]
# a2 = a2.drop_duplicates(['date', 'id_ves', 'id_vsd'], 'last')
# a2['first_dig'] = a2.volume.astype(str).str.slice(0, 2)

# df1 = a1.merge(a2, on=['id_ves', 'date', 'first_dig'], how='right')
# kilo_idx = df1[df1.unit == '\\N'].index
# df1.loc[kilo_idx, 'volume'] = df1.volume / 1000
# df1 = df1.dropna()
# df1['error'] = 1 - df1.catch_volume / df1.volume
# df1

<ipython-input-204-0d2333eaa64e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a1['first_dig'] = a1.catch_volume.astype(str).str.slice(0, 2)


,id_ves,date,catch_volume,id_own_x,first_dig,id_vsd,id_own_y,Product_period,date_vsd,volume,unit,fish,error
171,1284,2022-04-17,0.379500,1047.0,0.,7599412,1047,\N,2022-04-18 03:44:38,0.0,тонна,палтус черный свежий,-inf
172,1284,2022-04-17,0.028050,1047.0,0.,7599412,1047,\N,2022-04-18 03:44:38,0.0,тонна,палтус черный свежий,-inf
173,1284,2022-04-17,0.125400,1047.0,0.,7599412,1047,\N,2022-04-18 03:44:38,0.0,тонна,палтус черный свежий,-inf
174,1284,2022-04-17,0.768000,1047.0,0.,7599412,1047,\N,2022-04-18 03:44:38,0.0,тонна,палтус черный свежий,-inf
175,1284,2022-04-17,0.177480,1047.0,0.,7599412,1047,\N,2022-04-18 03:44:38,0.0,тонна,палтус черный свежий,-inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,1284,2022-01-01,0.091022,1047.0,0.,8511941,1047,\N,2022-01-02 05:13:01,0.0,тонна,зубатка свежая,-inf
1275,1284,2022-01-01,0.393763,1047.0,0.,7031948,1047,\N,2022-01-02 05:13:02,0.0,тонна,камбала,-inf
1276,1284,2022-01-01,0.681767,1047.0,0.,7031948,1047,\N,2022-01-02 05:13:02,0.0,тонна,камбала,-inf
1277,1284,2022-01-01,0.063296,1047.0,0.,7031948,1047,\N,2022-01-02 05:13:02,0.0,тонна,камбала,-inf


In [239]:
a1 = catch_simp[(catch_simp.id_ves == 1628) & (catch_simp.date==pd.to_datetime(datetime.date(2022, 3, 14)))]
a1['first_dig'] = a1.catch_volume.astype(str).str.slice(0, 2)

a1

<ipython-input-239-e7c822338680>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a1['first_dig'] = a1.catch_volume.astype(str).str.slice(0, 2)


,id_ves,date,catch_volume,id_own,first_dig
53244,1628,2022-03-14,31.449,961,31
53245,1628,2022-03-14,58.858,961,58


In [188]:
a2 = ext1_simp_mrg[(ext1_simp_mrg.id_ves == 1628) & (ext1_simp_mrg.date==pd.to_datetime(datetime.date(2022, 3, 14)))]
a2 = a2.drop_duplicates(['date', 'id_ves', 'id_vsd'])
a2['first_dig'] = a2.volume.astype(str).str.slice(0, 2)
a2

,id_vsd,id_ves,id_own,date,Product_period,date_vsd,volume,unit,fish,first_dig
1164446,7492878,1628,961,2022-03-14,\N,2022-03-15 01:18:03,58858.0,\N,минтай,58
1164448,9168097,1628,961,2022-03-14,\N,2022-03-15 01:18:02,31449.0,\N,минтай,31


In [191]:
a1.merge(a2, on=['id_ves', 'date','first_dig'], how='right')

,id_ves,date,catch_volume,id_own_x,first_dig,id_vsd,id_own_y,Product_period,date_vsd,volume,unit,fish
0,1628,2022-03-14,58.858,961,58,7492878,961,\N,2022-03-15 01:18:03,58858.0,\N,минтай
1,1628,2022-03-14,31.449,961,31,9168097,961,\N,2022-03-15 01:18:02,31449.0,\N,минтай


In [180]:
a1.merge(a2, on=['id_ves', 'date']).drop_duplicates(['date', 'id_ves', 'id_vsd'])

,id_ves,date,catch_volume,id_own_x,id_vsd,id_own_y,Product_period,date_vsd,volume,unit,fish
0,1628,2022-03-14,31.449,961,7492878,961,\N,2022-03-15 01:18:03,58858.0,\N,минтай
2,1628,2022-03-14,31.449,961,9168097,961,\N,2022-03-15 01:18:02,31449.0,\N,минтай


In [197]:
df1[df1.error > 0.1]

,id_ves,date,catch_volume,id_own_x,first_dig,id_vsd,id_own_y,Product_period,date_vsd,volume,unit,fish,error


In [12]:
set(catch_simp.id_ves) & set(product_simp.id_ves)

697

In [18]:
catch_prod_mrg = product_simp.merge(catch_simp, on='id_ves', how='left')

In [ ]:
catch_prod_mrg.head()

In [5]:
ext2.date_vsd = pd.to_datetime(ext2.date_vsd)

In [6]:
test_id = catch.id_own.iloc[0]

In [7]:
catch[(catch.id_own == test_id) & (catch.date.str.contains('2022-04-20'))]

,id_ves,date,id_region,id_fish,catch_volume,id_regime,permit,id_own
84248,2024908,2022-04-20,274,400,10.442,1,2351477,7115834
84249,2024908,2022-04-20,274,403,14.093,1,2351477,7115834
84250,2024908,2022-04-20,274,451,14.692,1,2351477,7115834
84251,2024908,2022-04-20,274,457,0.209,97,2351477,7115834
84252,3965387,2022-04-20,273,849,4.573,1,4877462,7115834
84296,8270286,2022-04-20,267,692,30.299,1,3454942,7115834
84309,2701764,2022-04-20,274,400,10.531,1,5925461,7115834
84310,2701764,2022-04-20,274,403,13.642,1,5925461,7115834
84311,2701764,2022-04-20,274,451,15.276,1,5925461,7115834


In [8]:
ext1[(ext1.id_own == test_id) & (ext1.date_fishery.str.contains('2022-04-20'))]

,id_fishery,id_own,date_fishery,numPart,id_Plat,id_vsd,Name_Plat,Product_period,Region_Plat
16689,7244716,7115834,2022-04-20 00:00:00,\N,7127846,326315,\N,\N,\N
16690,7244716,7115834,2022-04-20 00:00:00,\N,7127846,3527650,\N,\N,\N
16691,7244716,7115834,2022-04-20 00:00:00,\N,7127846,2233380,\N,\N,\N
16692,7244716,7115834,2022-04-20 00:00:00,\N,7127846,2912231,\N,\N,\N
16693,5868605,7115834,2022-04-20 00:00:00,\N,2403603,9466358,\N,\N,\N
16694,3487610,7115834,2022-04-20 00:00:00,\N,4899079,401394,\N,\N,\N
16729,867437,7115834,2022-04-20 00:00:00,\N,6996629,3734122,\N,\N,\N
16730,867437,7115834,2022-04-20 00:00:00,\N,6996629,4000204,\N,\N,\N
16731,867437,7115834,2022-04-20 00:00:00,\N,6996629,3338879,\N,\N,\N


In [9]:
ext1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3260798 entries, 0 to 3260797
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   id_fishery      int64 
 1   id_own          int64 
 2   date_fishery    object
 3   numPart         object
 4   id_Plat         int64 
 5   id_vsd          int64 
 6   Name_Plat       object
 7   Product_period  object
 8   Region_Plat     object
dtypes: int64(4), object(5)
memory usage: 223.9+ MB


In [10]:
test_vsd = ext1[ext1.id_own == test_id].id_vsd.iloc[0]
test_vsd

8482136

In [11]:
ext1[ext1.id_own == test_id]

,id_fishery,id_own,date_fishery,numPart,id_Plat,id_vsd,Name_Plat,Product_period,Region_Plat
9,608483,7115834,2022-05-24 00:00:00,\N,660693,8482136,\N,\N,\N
10,608483,7115834,2022-05-24 00:00:00,\N,660693,8373714,\N,\N,\N
11,608483,7115834,2022-05-24 00:00:00,\N,660693,7280720,\N,\N,\N
12,608483,7115834,2022-05-24 00:00:00,\N,660693,6399168,\N,\N,\N
13,4128905,7115834,2022-05-24 00:00:00,\N,8712885,436652,\N,\N,\N
...,...,...,...,...,...,...,...,...,...
3260475,608483,7115834,2021-01-01 00:00:00,\N,660693,8333076,\N,\N,\N
3260522,4605901,7115834,2020-12-31 00:00:00,\N,7619579,1578396,\N,\N,\N
3260794,3458281,7115834,2020-12-29 00:00:00,\N,906811,6111511,\N,\N,\N
3260795,3458281,7115834,2020-12-29 00:00:00,\N,906811,3138810,\N,\N,\N


In [12]:
ext2[ext2.id_vsd == test_vsd]

,id_vsd,num_vsd,id_fish,fish,date_vsd,volume,unit
3114848,8482136,1962367,400,минтай,2022-05-24 23:43:01,29756,\N


In [13]:
product[product.id_ves == 4524386].groupby('id_ves').sum()

,id_prod_designate,id_prod_type,prod_volume,prod_board_volume
id_ves,,,,
4524386,476,1345203,7886.943,81866.738


In [14]:
product[product.id_ves == 4524386]

,id_ves,date,id_prod_designate,id_prod_type,prod_volume,prod_board_volume
368,4524386,2022-01-01,1,3964,0.161,4.485
369,4524386,2022-01-01,1,3524,18.172,352.446
370,4524386,2022-01-01,1,2606,1.785,29.438
371,4524386,2022-01-01,1,2472,0.437,9.798
3398,4524386,2022-01-02,1,3964,0.598,5.083
...,...,...,...,...,...,...
341714,4524386,2022-04-07,1,1472,0.000,11.109
341715,4524386,2022-04-07,1,2472,0.000,4.922
341716,4524386,2022-04-07,1,2606,0.000,37.068
341717,4524386,2022-04-07,1,3524,0.000,916.570


In [15]:
product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395071 entries, 0 to 395070
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id_ves             395071 non-null  int64  
 1   date               395071 non-null  object 
 2   id_prod_designate  395071 non-null  int64  
 3   id_prod_type       395071 non-null  int64  
 4   prod_volume        395071 non-null  float64
 5   prod_board_volume  395071 non-null  float64
dtypes: float64(2), int64(3), object(1)
memory usage: 18.1+ MB


In [16]:
ext_merge = ext1.merge(ext2, on='id_vsd')

In [17]:
ext_merge_simp = ext1_simp.merge(ext2_simp, on='id_vsd')

In [18]:
a = list(catch.columns)
a[0] = 'id_fishery'
catch.columns = a

In [19]:
ext_merge_simp.shape

(3205586, 15)

In [20]:
ext_merge_simp = ext_merge_simp.merge(catch_simp, on='id_ves')

In [21]:
ext_merge_simp.head()

,id_ves,id_own_x,date_fishery,numPart,id_Plat,id_vsd,Name_Plat,Product_period,Region_Plat,num_vsd,...,date_vsd,volume,unit,date,id_region,id_fish_y,catch_volume,id_regime,permit,id_own_y
0,1657,1175,2022-05-24 00:00:00,\N,6292428,8660287,\N,\N,\N,3697783,...,2022-05-24 17:31:35,17522,\N,2022-01-23,264,292,4.497,1,2986,1175
1,1657,1175,2022-05-24 00:00:00,\N,6292428,8660287,\N,\N,\N,3697783,...,2022-05-24 17:31:35,17522,\N,2022-01-24,264,292,10.228,1,2986,1175
2,1657,1175,2022-05-24 00:00:00,\N,6292428,8660287,\N,\N,\N,3697783,...,2022-05-24 17:31:35,17522,\N,2022-01-25,264,292,11.133,1,2986,1175
3,1657,1175,2022-05-24 00:00:00,\N,6292428,8660287,\N,\N,\N,3697783,...,2022-05-24 17:31:35,17522,\N,2022-01-26,264,292,7.760,1,2986,1175
4,1657,1175,2022-05-24 00:00:00,\N,6292428,8660287,\N,\N,\N,3697783,...,2022-05-24 17:31:35,17522,\N,2022-01-27,264,292,2.139,1,2986,1175


In [23]:
ext_merge_simp.drop(columns=['numPart', 'Name_Plat', 'Product_period', 'Region_Plat']).head()

KeyboardInterrupt: 

In [ ]:
ext_merge_simp[['id_own_x', 'date_fishery', 'Product_period', 'date', 'catch_volume', 'volume']].head()

In [22]:
ext_merge_simp.columns


Index(['id_ves', 'id_own_x', 'date_fishery', 'numPart', 'id_Plat', 'id_vsd',
       'Name_Plat', 'Product_period', 'Region_Plat', 'num_vsd', 'id_fish_x',
       'fish', 'date_vsd', 'volume', 'unit', 'date', 'id_region', 'id_fish_y',
       'catch_volume', 'id_regime', 'permit', 'id_own_y'],
      dtype='object')